In [3]:
import requests #一些关键包
from urllib import request
from bs4 import BeautifulSoup
import time
import xlwt
import xlrd
import os
import pandas as pd
import numpy as np
import math
import re
def get_headers(): #得到伪造的文件头（考虑到验证问题，请自行采集cookie）
    headers = {
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
                'Accept-Language': 'zh-CN,zh;q=0.8',
                'Connection': 'keep-alive',
                'Accept-Encoding': 'gzip, deflate, br',
                'Cache-Control': 'max-age=0',
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36'
               }
    headers['Cookie']=''
    return headers
def GetMiddleStr(content,startStr,endStr): #对用户页面的HTML进行过滤
    startIndex = content.find(startStr)
    if startIndex>=0:
        startIndex += len(startStr)
    endIndex = content.find(endStr,startIndex)
    return content[startIndex:endIndex].replace("\\","")
def GetMiddleStr_(content,index): #对用户页面的HTML进行过滤
    find_str='<script>FM.view({"ns":"'
    find_str_='"html":"'
    end_str='})</script>'
    startIndex=0
    for i in range(index):
        startIndex = content.find(find_str,startIndex+1)
    startIndex=content.find(find_str_,startIndex)+len(find_str_)
    endIndex = content.find(end_str,startIndex)
    return content[startIndex:endIndex].replace("\\","")
def get_all_pages_link(keyword):  #得到当天一个关键词对应的全部热点页面链接
    links=[]
    url='https://s.weibo.com/weibo?q='+keyword+'&xsort=hot&suball=1&timescope=custom:2020-'+month+'-'+day+'-0:2020-'+month+'-'+day+'-23&Refer=g'
    first_page=requests.get(url,headers=get_headers()).text
    first_page_soup = BeautifulSoup(first_page, 'lxml')
    page_inf=first_page_soup.find_all('ul',class_="s-scroll")
    for each in page_inf[0].find_all('a'):
        links.append(each.get('href'))
    return links
def get_all_pages_link_by_date(keyword):  #按日期得到一个关键词对应的全部热点页面链接（临时编，推荐调用datetime模块）
    links=[]
    for i in range(60):
        time.sleep(2)
        print('正在收集第'+str(i+1)+'天页码链接')
        month='02'
        day=i+1
        if i+1>31and i+1<61:
            month='03'
            day=i+1-29
        elif i+1>=61:
            month='04'
            day=i+1-60
        else:
            pass
        if day<10:
            day='0'+str(day)
        else:
            day=str(day)
        url='https://s.weibo.com/weibo?q='+keyword+'&xsort=hot&suball=1&timescope=custom:2020-'+month+'-'+day+'-0:2020-'+month+'-'+day+'-23&Refer=g'
        first_page=requests.get(url,headers=get_headers()).text
        first_page_soup = BeautifulSoup(first_page, 'lxml')
        page_inf=first_page_soup.find_all('ul',class_="s-scroll")
        if len(page_inf)==0:
            links.append('weibo?q='+keyword+'&xsort=hot&suball=1&timescope=custom:2020-'+month+'-'+day+'-0:2020-'+month+'-'+day+'-23&Refer=g')
        else:
            for each in page_inf[0].find_all('a'):
                links.append(each.get('href'))
    return links
def get_user_info(link_inf):  #得到一条微博的用户信息
    user_inf_=''
    user_type=''
    user_types=['官方认证用户','普通用户','个人认证用户','企业认证用户','政府认证用户','明星、自媒体、博主等大V用户','明星、自媒体、博主等大V用户','明星、自媒体、博主等大V用户','明星、自媒体、博主等大V用户']
    type_num=['100206','other','100505','100606','100106','100406','100605','100306','103505']
    try_times=3
    if link_inf[0]=='u':
        link_inf=link_inf[2:]
    for n in range(0,len(user_types)):
        for i in range(try_times):
            try:
                if n==1:
                    user_inf=requests.get('https://weibo.com/u/'+link_inf+'?refer_flag=1001030103_&is_hot=1',headers=get_headers(),timeout=15)
                else:
                    user_inf=requests.get('https://weibo.com/p/'+type_num[n]+link_inf+'/home?from=page_'+type_num[n]+'&mod=TAB&is_hot=1',headers=get_headers(),timeout=15)
                if user_inf.status_code == 200:
                    break
            except:
                print('第'+str(i+1)+'次获取信息失败')
                if i==2:
                    print('已跳过，继续执行')
                    return 'null','null','null','null'
        user_inf=user_inf.text
        #print(user_inf)
        user_inf_=GetMiddleStr(str(user_inf),r'<script>FM.view({"ns":"","domid":"Pl_Core_T8CustomTriColumn__3","css":["style/css/module/pagecard/PCD_counter.css?version=bd9471367eec98f8"],"html":"',r'})</script>')
        if user_inf_=='':
            continue
        else:
            user_type=user_types[n]
            break
    if user_inf_=='':
        user_type='其他'
        return 'null','null','null',user_type
    user_inf_soup=BeautifulSoup(user_inf_,'lxml')
    forward=user_inf_soup.find_all('strong')[1].string
    weibo_num=user_inf_soup.find_all('strong')[2].string
    user_inf_=GetMiddleStr_(str(user_inf),5)
    user_inf_soup=BeautifulSoup(user_inf_,'lxml')
    if user_inf_=='':
        print('tag标签缺失')
        return forward,weibo_num,'null',user_type
    tag=user_inf_soup.find('div',class_='pf_intro').get('title')
    return forward,weibo_num,tag,user_type
def get_all_user_inf(link_inf,keyword):#得到全部的用户信息
    index=0
    fans=[]
    weibo_num=[]
    tags=[]
    users_type=[]
    link_index=link_inf
    norepeat=[]#去重
    np.array(range(0,len(link_inf)))
    norepeat.append(link_inf[0])
    for i in range(0,len(link_inf)):
        repeat_num=-1
        repeat=0
        each_link=link_inf[i]
        for j in range(0,len(norepeat)):
            if each_link==norepeat[j]:
                repeat_num=j
                repeat=1
        if repeat==1:
            link_index[i]=repeat_num
        else:
            norepeat.append(each_link)
            link_index[i]=len(norepeat)-1 
    #print(len(norepeat))
    print('开始逐条获取用户信息('+keyword+')...共',len(norepeat),'条')
    for each in norepeat:
        time.sleep(1)
        index=index+1
        print('正在采集第',index,'个用户信息')
        if index%100==0:
            print(fans,weibo_num,tags,users_type)
        try:
            fan,weibo_num_,tag,user_type=get_user_info(str(each))
        except BaseException:
            print('出现错误，共收集',index,'条,如下所示：')
            print(fans,weibo_num,tags,users_type)
            fan,weibo_num_,tag,user_type='null','null','null','null'
            time.sleep(20)
            print('已跳过，继续执行')
        fans.append(fan)
        weibo_num.append(weibo_num_)
        tags.append(tag)
        users_type.append(user_type)
    print('用户信息采集完成')
    workbook=xlwt.Workbook(encoding='utf-8')#把原始信息输出到excel
    booksheet=workbook.add_sheet('Sheet 1', cell_overwrite_ok=True)
    for i in range(0,len(link_inf)):
        booksheet.write(i, 0, fans[link_index[i]])
        booksheet.write(i, 1, weibo_num[link_index[i]])
        booksheet.write(i, 2, tags[link_index[i]])
        booksheet.write(i, 3, users_type[link_index[i]])
    workbook.save('用户信息'+keyword+str(time.strftime('%Y-%m-%d',time.localtime(time.time())))+'.xls')
    print('用户信息录入成功')
def get_keyword_all_inf(page_links,keyword): #得到当天一个关键词下的全部信息并输出到excel
    index=0
    index_=0
    start_time=time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()))
    inf=[]
    names=[]
    times=[]
    contents=[]
    forwards=[]
    comments=[]
    likes=[]
    link_inf=[]
    page=[]
    for each in page_links:
        time.sleep(1)
        weibo_inf=requests.get('https://s.weibo.com/'+each,headers=get_headers()).text
        index_=index_+1
        print('已收集页数：',index_)
        inf.append(weibo_inf)
    print('已获取所有微博内容')
    print('开始逐条处理信息...')
    for n in range(0,len(inf)):
        full_num=0
        str_=[]
        full_str=[]
        contents_list=[]
        full=[]
        weibo_inf_soup=BeautifulSoup(inf[n], 'lxml')
        num=len(weibo_inf_soup.find_all('div',class_='card-feed'))
        names_list=weibo_inf_soup.find_all('a',class_='name')
        times_list=weibo_inf_soup.find_all('p',class_='from')
        forwards_list=weibo_inf_soup.find_all('a',attrs={"action-type": "feed_list_forward"})
        comments_list=weibo_inf_soup.find_all('a',attrs={"action-type": "feed_list_comment"})
        likes_list=weibo_inf_soup.find_all('a',attrs={"action-type": "feed_list_like"})
        contents_list_=weibo_inf_soup.find_all('p',attrs={"class":"txt","node-type": "feed_list_content"})
        contents_list_full=weibo_inf_soup.find_all('p',attrs={"class":"txt","node-type": "feed_list_content_full"})
        for each in contents_list_:
            str_.append(each.get_text())
        for each in contents_list_full:
            full_str.append(each.get_text())
        for e in range(len(str_)):
            f=0
            cont=str_[e]
            cont=cont[:-10]
            for r in range(full_num,len(full_str)):
                compared=full_str[r]
                if cont in compared:
                    f=1
                    full_num=full_num+1
                    full.append(r)
                    break
                else:
                    continue
            if f==0:
                full.append(-1)
        for o in range(len(full)):
            if full[o]==-1:
                contents_list.append(str_[o])
            else:
                contents_list.append(full_str[full[o]])
        for i in range(0,num):
            index=index+1
            print(index,names_list[i].string)
            names.append(names_list[i].string)
            times.append(times_list[i].a.string)
            contents.append(contents_list[i])
            forwards.append(forwards_list[i].string)
            comments.append(comments_list[i].string)
            likes.append(likes_list[i].em.string)
            link_inf.append(GetMiddleStr(names_list[i].get('href'),'com/','?'))
            page.append(int(n+1))
    workbook=xlwt.Workbook(encoding='utf-8')#把原始信息输出到excel
    booksheet=workbook.add_sheet('Sheet 1', cell_overwrite_ok=True)
    for i in range(0,len(names)):
        booksheet.write(i, 0, names[i])
        booksheet.write(i, 1, contents[i])
        booksheet.write(i, 2, times[i])
        booksheet.write(i, 3, forwards[i])
        booksheet.write(i, 4, str(comments[i]))
        booksheet.write(i, 5, likes[i])
        booksheet.write(i, 6, str(start_time))
        booksheet.write(i, 7, link_inf[i])
        booksheet.write(i, 8, page[i])
    workbook.save(keyword+'内容信息'+str(time.strftime('%Y-%m-%d',time.localtime(time.time())))+'.xls')
    print('微博内容录入成功')
    return link_inf
def get_keyword_all_inf_by_date(page_links,keyword): #按日期得到一个关键词下的全部信息并输出到excel
    index=0
    index_=0
    start_time=time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()))
    names=[]
    times=[]
    contents=[]
    forwards=[]
    comments=[]
    likes=[]
    link_inf=[]
    page=[]
    symbols=[]
    for each in page_links:
        time.sleep(1)
        weibo_inf=requests.get('https://s.weibo.com/'+each,headers=get_headers()).text
        index=index+1
        print('已收集页数：',index)
        full_num=0
        str_=[]
        full_str=[]
        contents_list=[]
        full=[]
        weibo_inf_soup=BeautifulSoup(weibo_inf, 'lxml')
        num=len(weibo_inf_soup.find_all('div',class_='card-feed'))
        names_list=weibo_inf_soup.find_all('a',class_='name')
        times_list=weibo_inf_soup.find_all('p',class_='from')
        forwards_list=weibo_inf_soup.find_all('a',attrs={"action-type": "feed_list_forward"})
        comments_list=weibo_inf_soup.find_all('a',attrs={"action-type": "feed_list_comment"})
        likes_list=weibo_inf_soup.find_all('a',attrs={"action-type": "feed_list_like"})
        contents_list_=weibo_inf_soup.find_all('p',attrs={"class":"txt","node-type": "feed_list_content"})
        contents_list_full=weibo_inf_soup.find_all('p',attrs={"class":"txt","node-type": "feed_list_content_full"})
        for each in contents_list_:
            str_.append(each.get_text())
        for each in contents_list_full:
            full_str.append(each.get_text())
        for e in range(len(str_)):
            f=0
            cont=str_[e]
            cont=cont[:-10]
            for r in range(full_num,len(full_str)):
                compared=full_str[r]
                if cont in compared:
                    f=1
                    full_num=full_num+1
                    full.append(r)
                    break
                else:
                    continue
            if f==0:
                full.append(-1)
        for o in range(len(full)):
            if full[o]==-1:
                contents_list.append(str_[o])
            else:
                contents_list.append(full_str[full[o]])
        for i in range(0,num):
            index_=index_+1
            names.append(names_list[i].string)
            times.append(times_list[i].a.string)
            contents.append(contents_list[i])
            forwards.append(forwards_list[i].string)
            comments.append(comments_list[i].string)
            likes.append(likes_list[i].em.string)
            link_inf.append(GetMiddleStr(names_list[i].get('href'),'com/','?'))
            symbols.append(times_list[i].a.get('href'))
            page.append(index_)
    workbook=xlwt.Workbook(encoding='utf-8')#把原始信息输出到excel
    booksheet=workbook.add_sheet('Sheet 1', cell_overwrite_ok=True)
    for i in range(0,len(names)):
        booksheet.write(i, 0, names[i])
        booksheet.write(i, 1, contents[i])
        booksheet.write(i, 2, times[i])
        booksheet.write(i, 3, forwards[i])
        booksheet.write(i, 4, str(comments[i]))
        booksheet.write(i, 5, likes[i])
        booksheet.write(i, 6, str(start_time))
        booksheet.write(i, 7, link_inf[i])
        booksheet.write(i, 8, page[i])
        booksheet.write(i, 9, symbols[i])
    workbook.save(keyword+'内容信息'+str(time.strftime('%Y-%m-%d',time.localtime(time.time())))+'.xls')
    print('微博内容录入成功')
#print(get_user_info('1887344341'))#调试用
#print(get_keyword_all_inf(['/weibo?q=%E6%96%B0%E5%9E%8B%E5%86%A0%E7%8A%B6%E7%97%85%E6%AF%92&xsort=hot&Refer=hotmore&page=1'],'ss'))#调试用
def main_func(): #主函数，控制要找的关键词（当天，收集用户信息）
    keywords=['复产','复工','消费']
    all_link=[]
    print('收集开始...')
    for each in keywords:
        print('当前关键词：'+each)
        all_link.append(get_keyword_all_inf(get_all_pages_link(each),each))
    for i in range(0,len(keywords)):
        get_all_user_inf(all_link[i],keywords[i])
    print('已完成全部信息收集')
def main_func_(): #主函数，控制要找的关键词（按日期，不收集用户信息）
    keywords=['消费','复工','复产']
    all_link=[]
    print('收集开始...')
    for each in keywords:
        print('当前关键词：'+each)
        get_keyword_all_inf_by_date(get_all_pages_link_by_date(each),each)
    print('已完成全部信息收集')
main_func_()

收集开始...
当前关键词：消费
正在收集第1天页码链接
正在收集第2天页码链接
正在收集第3天页码链接
正在收集第4天页码链接
正在收集第5天页码链接
正在收集第6天页码链接
正在收集第7天页码链接
正在收集第8天页码链接
正在收集第9天页码链接
正在收集第10天页码链接
正在收集第11天页码链接
正在收集第12天页码链接
正在收集第13天页码链接
正在收集第14天页码链接
正在收集第15天页码链接
正在收集第16天页码链接
正在收集第17天页码链接
正在收集第18天页码链接
正在收集第19天页码链接
正在收集第20天页码链接
已收集页数： 1
已收集页数： 2
已收集页数： 3
已收集页数： 4
已收集页数： 5
已收集页数： 6
已收集页数： 7
已收集页数： 8
已收集页数： 9
已收集页数： 10
已收集页数： 11
已收集页数： 12
已收集页数： 13
已收集页数： 14
已收集页数： 15
已收集页数： 16
已收集页数： 17
已收集页数： 18
已收集页数： 19
已收集页数： 20
已收集页数： 21
已收集页数： 22
已收集页数： 23
已收集页数： 24
已收集页数： 25
已收集页数： 26
已收集页数： 27
已收集页数： 28
已收集页数： 29
已收集页数： 30
已收集页数： 31
已收集页数： 32
已收集页数： 33
已收集页数： 34
已收集页数： 35
已收集页数： 36
已收集页数： 37
已收集页数： 38
已收集页数： 39
已收集页数： 40
已收集页数： 41
已收集页数： 42
已收集页数： 43
已收集页数： 44
已收集页数： 45
已收集页数： 46
已收集页数： 47
已收集页数： 48
已收集页数： 49
已收集页数： 50
已收集页数： 51
已收集页数： 52
已收集页数： 53
已收集页数： 54
已收集页数： 55
已收集页数： 56
已收集页数： 57
已收集页数： 58
已收集页数： 59
已收集页数： 60
已收集页数： 61
已收集页数： 62
已收集页数： 63
已收集页数： 64
已收集页数： 65
已收集页数： 66
已收集页数： 67
已收集页数： 68
已收集页数： 69
已收集页数： 70
已收集页数： 71
已收集页数： 72
已收集页数： 73
已收集页数： 74
已